In [70]:
import pandas as pd
# read data
df = pd.read_csv("stage3.csv")
ownership_df = pd.read_csv("gunownership.csv")
# population_df = pd.read_csv("gun_ownership.csv")

### Gun incidents per year

In [71]:
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
incidents_per_year = df['year'].value_counts()

aa = df[df['state'] == 'Illinois']
bb = aa[aa['incident_characteristics'].str.contains('Mass') == True]
cc = sorted(bb['year'].unique())

for thing in [2014, 2015]:
    if thing in bb['year'].unique():
        print("yes")
        print(bb[bb['year'] == thing]['incident_id'].count())
    else:
        print("no")



# not many incidents from 2013 were recorded (incomplete data) -> removed from set

yes
26
yes
25


### Gun incidents per month (per year)

In [72]:
# for each year, counts number of incidents per year
df['month'] = df['date'].dt.month

### Incidents per state

In [73]:
year_per_state = df.groupby(['year', 'state']).size()

In [74]:
incidents_per_state = df.groupby(['state']).size()

In [75]:
# import stuff
from bokeh.plotting import figure
from bokeh.io import output_file, show, save
from bokeh.models import DataRange1d, FactorRange, ColumnDataSource, Plot, HBar, LabelSet
import pandas as pd
import numpy as np

### Some data for plots

In [76]:
# add number of incidents per state
incidents = df.groupby(['state']).size()
incidents = [incident for incident in incidents]
ownership_df['incidents'] = incidents

In [77]:
years_df = pd.DataFrame()
years_df['year'] = ['2014', '2015', '2016', '2017', '2018']
years_df['incidents'] = [number for number in df.groupby('year').size()]

# fill in columns for incidents per yer PER STATE
ownership_df['2014'] = [number for number in year_per_state[:51]]
ownership_df['2015'] = [number for number in year_per_state[51:102]]
ownership_df['2016'] = [number for number in year_per_state[102:153]]
ownership_df['2017'] = [number for number in year_per_state[153:204]]
ownership_df['2018'] = [number for number in year_per_state[204:]]

# incidents per month in the United States
incidents_per_month = [number for number in df.groupby(['year', 'month']).size()]


### Zooming in: analysing cities

In [78]:
df_2014 = df.loc[df['year'] == 2014]
cities_2014 = df_2014['city_or_county'].value_counts()[:10]

df_2015 = df.loc[df['year'] == 2015]
cities_2015 = df_2015['city_or_county'].value_counts()[:10]

df_2016 = df.loc[df['year'] == 2016]
cities_2016 = df_2016['city_or_county'].value_counts()[:10]

df_2017 = df.loc[df['year'] == 2017]
cities_2017 = df_2017['city_or_county'].value_counts()[:10]

df_2018 = df.loc[df['year'] == 2018]
cities_2018 = df_2018['city_or_county'].value_counts()[:10]



In [79]:
# violent_incidents = df.loc[(df['n_killed'] > 0) | (df['n_injured'] > 0)]
violent_incidents = df[df['year'] < 2018]

violent_incidents['city_or_county'].value_counts()[:10]


Chicago         10298
Baltimore        3703
Washington       3095
New Orleans      2926
Philadelphia     2824
Jacksonville     2372
Saint Louis      2351
Houston          2331
Milwaukee        2294
Memphis          2283
Name: city_or_county, dtype: int64

In [80]:
a = df.loc[(df['n_killed'] > 0) | (df['n_injured'] > 0)]
a = a[a['year'] < 2018]
a['city_or_county'].value_counts()[:10]

Chicago         9618
Baltimore       2794
Philadelphia    2451
New Orleans     1938
Saint Louis     1932
Memphis         1732
Houston         1698
Milwaukee       1620
Columbus        1473
Washington      1450
Name: city_or_county, dtype: int64

In [81]:
b = df[df['year'] < 2018]

c = b.groupby('city_or_county').sum()
c.sort_values('n_killed', ascending=False)[:10]

,incident_id,n_killed,n_injured,latitude,longitude,year,month
city_or_county,,,,,,,
Chicago,5600947928,2011,9924,417641.1118,-875432.6540,20757127,70479
Houston,1168747818,1063,1143,67480.0208,-215715.7263,4698039,15109
Baltimore,2021057446,1003,2289,143699.5605,-280147.0074,7463930,25308
Saint Louis,1242462088,865,1657,90532.3711,-211360.3372,4738603,16076
Philadelphia,1333568644,854,2190,110408.4987,-208170.5559,5691132,18254
New Orleans,1479706167,658,1778,84319.5052,-253320.5941,5897339,18873
Los Angeles,477275952,608,524,34153.0812,-118767.9580,2045532,6563
Memphis,1254733240,587,1607,77596.6462,-198802.4818,4601806,15085
Indianapolis,910340237,587,1079,69725.6922,-150921.5920,3684264,12028


In [82]:
df['state'].value_counts()[:10]

Illinois          17538
California        16267
Florida           15016
Texas             13566
Ohio              10234
New York           9696
Georgia            8919
Pennsylvania       8909
North Carolina     8727
Louisiana          8094
Name: state, dtype: int64

In [51]:
import numpy as np
from bokeh.plotting import figure
from bokeh.io import output_notebook, push_notebook, show
import pandas

from bokeh.models import DataRange1d
from bokeh.models import PanTool, ResetTool, WheelZoomTool, HoverTool, LassoSelectTool, BoxSelectTool, SaveTool
from bokeh.models import ColumnDataSource

counts = [2283, 2294, 2331, 2351, 2372, 2824, 2926, 3095, 3703, 10298]
cities = ['Memphis', 'Milwaukee', 'Houston', 'Saint Louis', 'Jacksonville', 'Philadelphia', 'New Orleans', 'Washington', 'Baltimore', 'Chicago']

p = figure(y_range=cities, title='Cities with most gun incidents 2014 - 2017', plot_width=600, plot_height=270)

source = ColumnDataSource(data=dict(x=counts, y=cities))

hover = HoverTool(tooltips=[
    ("City", "@y"),
    ("Amount", "@x"),
])

p.toolbar.tools = [ResetTool(), hover]

p.hbar(height=0.6, y="y", right="x", left=0, color="#F55F60", source=source)

p.xaxis.major_label_text_font_size = '14px'
p.yaxis.major_label_text_font_size = '14px'
p.title.text_font_size = "16px"
p.title.align = "center"
p.outline_line_color = None
p.toolbar.logo = None
p.toolbar_location = None

output_file("way-too-many-incidents.html")
save(p)

'C:\\Users\\Marijn\\Desktop\\project\\DataProcessing\\way-too-many-incidents.html'

In [52]:
counts3 = [560, 587, 587, 608, 658, 854, 865, 1003, 1063, 2011]
cities3 = ['Detroit', 'Indianapolis', 'Memphis', 'Los Angeles', 'New Orleans', 'Philadelphia', 'Saint Louis', 'Baltimore', 'Houston', 'Chicago']

p3 = figure(y_range=cities3, title='Cities with most gun deaths 2014 - 2017', plot_width=600, plot_height=270)

source = ColumnDataSource(data=dict(x=counts3, y=cities3))

hover = HoverTool(tooltips=[
    ("City", "@y"),
    ("Amount", "@x"),
])

p3.toolbar.tools = [ResetTool(), hover]

p3.hbar(height=0.6, y='y', right='x', left=0, color="#F55F60", source=source)

#labels = LabelSet(x='x', y='y', text='x', level='glyph', source=source, render_mode='canvas')

p3.xaxis.major_label_text_font_size = '14px'
p3.yaxis.major_label_text_font_size = '14px'
p3.title.text_font_size = "16px"
p3.title.align = "center"
p3.outline_line_color = None
p3.toolbar.logo = None
p3.toolbar_location = None

output_file("way-too-many-incidents.html")
save(p3)

'C:\\Users\\Marijn\\Desktop\\project\\DataProcessing\\way-too-many-incidents.html'

In [53]:
output_file("way-too-many-incidents.html")

counts2 = [1450, 1473, 1620, 1698, 1732, 1932, 1938, 2451, 2794, 9618]
cities2 = ['Washington', 'Columbus', 'Milwaukee', 'Houston', 'Memphis', 'Saint Louis', 'New Orleans', 'Philadelphia', 'Baltimore', 'Chicago']

p2 = figure(y_range=cities2, title='Top 10 Cities with Most Gun Incidents 2014-2017 (n hurt > 0)')
p2.hbar(height=0.6, y=cities2, right=counts, left=0)
p2.xaxis.major_label_text_font_size = '12pt'
p2.xaxis.axis_label_text_font_style = 'bold'
p2.yaxis.axis_label_text_font_style = 'bold'
p2.yaxis.major_label_text_font_size = '12pt'
p2.title.align = 'center'
save(p2)

'C:\\Users\\Marijn\\Desktop\\project\\DataProcessing\\way-too-many-incidents.html'

### Number of registered guns and incidents (per state)

In [54]:
#Define the output file path
output_file("way-too-many-incidents.html")

#Create the figure object
f = figure()
f.x_range=DataRange1d(start=0)
f.y_range=DataRange1d(start=0)

## Add a Title to the plot (We essentially use css properties here)
# You can find all function of f.title, using dir(f.title)
f.title.text="Registered Guns and Incidents"
f.title.text_font_size="2px"
f.title.align="center"

## Add some axis information (after all, a plot without axis descriptions is nothing more than abstract art)
f.xaxis.axis_label="Number of Registered Guns"
f.yaxis.axis_label="Number of Incidents"

#adding glyphs
f.circle(x=ownership_df['guns_registered'], y=ownership_df['incidents'])

#Save and show the figure
save(f)

'C:\\Users\\Marijn\\Desktop\\project\\DataProcessing\\way-too-many-incidents.html'

### Number of registered guns per capita and incidents (per state)

In [55]:
#Define the output file path
output_file("way-too-many-incidents.html")

#Create the figure object
f2 = figure()

f2.x_range=DataRange1d(start=0, end=250)
f2.y_range=DataRange1d(start=0, end=20000)

## Add a Title to the plot (We essentially use css properties here)
# You can find all function of f.title, using dir(f.title)
f2.title.text="Guns per Capita and Incidents"
f2.title.text_font_size="25px"
f2.title.align="center"

## Add some axis information (after all, a plot without axis descriptions is nothing more than abstract art)
f2.xaxis.axis_label="Guns per Capita"
f2.yaxis.axis_label="Incidents"

#adding glyphs
f2.circle(x=ownership_df['guns_per_capita'], y=ownership_df['incidents'])

#Save and show the figure
save(f2)

'C:\\Users\\Marijn\\Desktop\\project\\DataProcessing\\way-too-many-incidents.html'

### Incidents per year (entire country)

In [116]:
from bokeh.models import DataRange1d

f3 = figure(y_range=(0, 65000), plot_width=500, plot_height=500)

f3.title.text="Gun incidents per year"
f3.title.text_font_size="16px"
f3.title.align="center"

data = {"year" : [i for i in years_df['year']], "incidents" : [i for i in years_df['incidents']]}
year_df = pd.DataFrame(data, index = ['2014', '2015', '2016', '2017', '2018'])
year_df = year_df.drop('2018')

f3.line(x=year_df['year'], y=year_df['incidents'], color="#F55F60", line_width=10)

f3.xaxis[0].major_label_text_font_size="14px"
f3.yaxis[0].major_label_text_font_size="14px"
f3.xaxis[0].axis_label_text_font_size="14px"
f3.yaxis[0].axis_label_text_font_size="14px"
f3.outline_line_color = None
f3.toolbar.logo = None
f3.toolbar_location = None

f3.xaxis.ticker = [2014, 2015, 2016, 2017]
f3.toolbar.tools = [ResetTool()]

output_file("way-too-many-incidents.html")
save(f3)

'C:\\Users\\Marijn\\Desktop\\project\\DataProcessing\\way-too-many-incidents.html'

### Fatal incidents per year (entire country)

In [20]:
# locate all incidents where people have been killed
deadly_incidents_per_year = [number for number in df.loc[df['n_killed'] > 0].groupby('year').size()]

#Define the output file path
output_file("way-too-many-incidents.html")

#Create the figure object
f7 = figure()

## Add a Title to the plot (We essentially use css properties here)
# You can find all function of f.title, using dir(f.title)
f7.title.text="Fatal Gun Incidents in the United States"
f7.title.text_font_size="25px"
f7.title.align="center"

## Add some axis information (after all, a plot without axis descriptions is nothing more than abstract art)
f7.xaxis.axis_label="Year"
f7.yaxis.axis_label="Incidegnts"

#adding glyphsg
f7.circle(x=years_df['year'], y=deadly_incidents_per_year)

#Save and show the figure
save(f7)

'/home/hughmee/Documents/testfordav/rest/way-too-many-incidents.html'

### Number of people killed each year (entire country)

In [21]:
# get number of people killed each year
per_year = df.groupby('year').sum()
killed_per_year = [killed for killed in per_year['n_killed']]

output_file("way-too-many-incidents.html")
f8 = figure()

# add title
f8.title.text="Gun Deaths in the United States"
f8.title.text_font_size="25px"
f8.title.align="center"

## add some axis information
f8.xaxis.axis_label="Year"
f8.yaxis.axis_label="Killed"

# adding glyphsg
f8.circle(x=years_df['year'], y=killed_per_year)

# save and show the figure
save(f8)

'/home/hughmee/Documents/testfordav/rest/way-too-many-incidents.html'

### Incidents per month (entire country)

In [22]:
years = ['2014', '2015', '2016', '2017', '2018']
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

x_data = [(year, month) for year in years for month in months][:51]
monthly = [number for number in df.groupby(['year', 'month']).size()]

#Define the output file path
output_file("way-too-many-incidents.html")

#Create the figure object
f4 = figure(x_range=FactorRange(*x_data), plot_width=800, plot_height=800)

## Add a Title to the plot
f4.title.text="Gun Incidents in the United States (per Month)"
f4.title.text_font_size="25px"
f4.title.align="center"

## Add some axis information (after all, a plot without axis descriptions is nothing more than abstract art)
f4.xaxis.axis_label="Year-Month"
f4.yaxis.axis_label="Number of Incidents"

#adding glyphsg
f4.vbar(x=x_data, top=monthly, width=1, fill_color="#77b7f1")

f4.y_range.start = 0
f4.x_range.range_padding = 0
f4.xaxis.major_label_orientation = 1
#f4.xgrid.grid_line_color = None

#Save and show the figure
save(f4)


'/home/hughmee/Documents/testfordav/rest/way-too-many-incidents.html'

### Mass Shootings per Month (Entire Country)

In [23]:
mass_shootings = df[df['incident_characteristics'].str.contains("Mass Shooting") == True]

years = ['2014', '2015', '2016', '2017', '2018']
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

x_data2 = [(year, month) for year in years for month in months][:51]
monthly2 = [number for number in mass_shootings.groupby(['year', 'month']).size()]

#Define the output file path
output_file("way-too-many-incidents.html")

#Create the figure object
f6 = figure(x_range=FactorRange(*x_data2), plot_width=800, plot_height=800)

## Add a Title to the plot
f6.title.text="Mass Shootings in the United States (per Month)"
f6.title.text_font_size="25px"
f6.title.align="center"

## Add some axis information
f6.xaxis.axis_label="Year-Month"
f6.yaxis.axis_label="Number of Incidents"

#adding glyphsg
f6.vbar(x=x_data2, top=monthly2, width=1, fill_color="#f1c177")

f6.y_range.start = 0
f6.x_range.range_padding = 0
f6.xaxis.major_label_orientation = 1
f6.xgrid.grid_line_color = None

#Save and show the figure
save(f6)


'/home/hughmee/Documents/testfordav/rest/way-too-many-incidents.html'

In [24]:
'''
import pandas as pd
from bokeh.palettes import Spectral4
from bokeh.plotting import figure, output_file, show
from bokeh.sampledata.stocks import AAPL, IBM, MSFT, GOOG


p = figure(plot_width=800, plot_height=250, x_axis_type="datetime")
p.title.text = 'Click on legend entries to hide the corresponding lines'

for data, name, color in zip([AAPL, IBM, MSFT, GOOG], ["AAPL", "IBM", "MSFT", "GOOG"], Spectral4):
    df = pd.DataFrame(data)
    df['date'] = pd.to_datetime(df['date'])
    p.line(df['date'], df['close'], line_width=2, color=color, alpha=0.8, legend=name)

p.legend.location = "top_left"
p.legend.click_policy="hide"

output_file("interactive_legend.html", title="interactive_legend.py example")

save(p)
'''

'\nimport pandas as pd\nfrom bokeh.palettes import Spectral4\nfrom bokeh.plotting import figure, output_file, show\nfrom bokeh.sampledata.stocks import AAPL, IBM, MSFT, GOOG\n\n\np = figure(plot_width=800, plot_height=250, x_axis_type="datetime")\np.title.text = \'Click on legend entries to hide the corresponding lines\'\n\nfor data, name, color in zip([AAPL, IBM, MSFT, GOOG], ["AAPL", "IBM", "MSFT", "GOOG"], Spectral4):\n    df = pd.DataFrame(data)\n    df[\'date\'] = pd.to_datetime(df[\'date\'])\n    p.line(df[\'date\'], df[\'close\'], line_width=2, color=color, alpha=0.8, legend=name)\n\np.legend.location = "top_left"\np.legend.click_policy="hide"\n\noutput_file("interactive_legend.html", title="interactive_legend.py example")\n\nsave(p)\n'